*Last edit: 21st August 2024*

### Estimation Error Matrix 

Incorporate robust optimization with estimation error matrix. We start by setting $\Xi = I$. We also want to take $\Xi = \text{diag}(\Sigma)$ and $\Xi = \text{diag}(\frac{1}{\sigma_i^2})$.

We take $N=24$ for monthly data and $N=100$ for daily data

When $\Xi$ is diagonal, we are saying that the errors for the different assets are independent.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import seaborn as sns; sns.set_theme()

from functions import *

import gurobipy as gp
from gurobipy import GRB

import os
import json
import re

In [2]:
save = True
heuristic = True

seed = 339
num_runs = 10000

for data in ['GICS','Fama-French']:
    if data == 'GICS':
        unit_list = ['monthly']
        num_sectors_list = [11]
    else:
        unit_list = ['monthly','daily']
        num_sectors_list = [5,10,12,17]
    for units in unit_list:
        for num_sectors in num_sectors_list:

            save_units = units.capitalize()
            
            if units == 'daily':
                N = [100]
                num_years=10
                num_units=int(num_years*365*(5/7))
            elif units=='monthly':
                N = [24]
                num_years=30
                num_units=int(num_years*12)
            
            true_mu, true_cov = true_mean_cov(data,units,num_sectors,num_units)
            
            np.random.seed(seed=seed)
            
            mu_hat = {}
            
            for run in range(num_runs):
                mu_hat[run] = {}
                for n in N:
                    mu_hat[run][n] = np.mean(simulate(true_mu, true_cov, N), axis=0)
            
            print('True Mean:')
            display(pd.DataFrame(true_mu).T.round(3))
            
            print('True Covariance:')
            display(pd.DataFrame(true_cov).round(3))
            
            if data == 'Fama-French':
                path = os.path.join('Simulated Robust','Fama-French',str(num_sectors)+' Sectors',save_units)
            elif data == 'GICS':
                path = os.path.join('Simulated Robust','GICS')
            
            identity = 0
            sigma = 0
            inv_sigma = 1
            
            assert identity + sigma + inv_sigma == 1, 'Choose exactly one type of estimation error matrix'
            
            if sigma:
                # topx = num_sectors // 2
                # bottomx = (num_sectors // 2) + 1
                topx = num_sectors
                bottomx = 0
                power = 2
            elif inv_sigma:
                topx = num_sectors
                bottomx = 0
                power = 25
                
            if sigma or inv_sigma:
                
                power_label = {0:'Zeroth',1:'',2:'Squared',3:'Cubed',4:'Fourth',5:'Fifth',10:'Tenth',
                               12:'Twelfth',15:'Fifteenth',25:'TwentyFifth'}[power]
                
                diag = np.diag(true_cov)
                topindices = np.argpartition(diag, -topx)[-topx:]
                bottomindices = np.argpartition(diag, bottomx)[:bottomx]
            
            Xi = np.eye(num_sectors)
            
            if identity:
                topx = 'NA'
                path = os.path.join(path,'Xi_identity') 
            elif sigma:
                path = os.path.join(path,'Xi_Sigma'+str(power_label))
                for idx in topindices:
                    Xi[idx][idx] = (diag[idx])**(power/2)
                for idx in bottomindices:
                    Xi[idx][idx] = np.mean((diag)**(power/2))
            elif inv_sigma:
                path = os.path.join(path,'Xi_invSigma'+str(power_label))
                for idx in topindices:
                    Xi[idx][idx] = 1/(diag[idx]**(power/2))
                for idx in bottomindices:
                    Xi[idx][idx] = 1/np.mean((diag)*(power/2))
            
            print('Xi:')
            display(pd.DataFrame(Xi))

            for lb_ub in [(1,3),(2,4),(3,5)]:
            
                if not heuristic:
                    if units == 'monthly':
                        if data == 'GICS':
                            if identity:
                                kappa_list = [0.5,1,5]
                            elif sigma:
                                kappa_list = [0.1,0.5,1]
                            elif inv_sigma:
                                # kappa_list = [x * (2**(power-1)) for x in [y*(power-3) if y*(power-3) >= 1 else y for y in [2.5,5,10]]]
                                if power == 1 or power == 2:
                                    kappa_list = [2.5,5,10]
                                elif power == 4:
                                    kappa_list = [20,40,100,125,150]
                                elif power == 10:
                                    kappa_list = [8960,17920,35840]
                        else:
                            if identity:
                                kappa_list = [0.5,0.75,2.5]
                            elif sigma:
                                kappa_list = [0.05,0.25,0.5]
                            elif inv_sigma:
                                # kappa_list = [x * (2**(power-1)) for x in [y*(power-3) if y*(power-3) >= 1 else y for y in [1,2.5,5]]]
                                if power == 1:
                                    kappa_list = [1,2.5,5]
                                elif power == 2:
                                    kappa_list = [2.5,5,10]
                                elif power == 4:
                                    kappa_list = [10,20,50,75,100]
                                elif power == 10:
                                    kappa_list = [3584,8960,17920]
                    elif units == 'daily':
                        if identity:
                            kappa_list = [0.05, 0.1, 0.25]
                        elif sigma:
                            kappa_list = [0.01,0.05,0.1]
                        elif inv_sigma:
                            # kappa_list = [x * (2**(power-7)) for x in [y*(power-8) if y*(power-8) >= 1 else y for y in [0.05, 0.1, 0.25]]]
                            if power == 1:
                                kappa_list = [0.05,0.1,0.25]
                            elif power == 2:
                                kappa_list = [0.05,0.1,0.25]
                            elif power == 4:
                                kappa_list = [0.05,0.1,0.25,0.33,0.5]
                            elif power == 10:
                                kappa_list = [0.4,0.8,2]
                    ratio = 'NA'
                else:
                    kappa_list = ['heuristic']
                    lb = lb_ub[0]
                    ub = lb_ub[1]
                    ratio = int((ub+lb)/2)
                    print('lb:',lb, 'ub:',ub, 'ratio:',ratio)
                
                print('List of Kappas:',kappa_list)
                
                if save:
                    incumbent_path = []
                    for directory in ['plots'] + os.path.normpath(path).split(os.sep):# + ['Zoomed']:
                        incumbent_path.append(directory)
                        if not os.path.exists(os.path.join(*incumbent_path)):
                            os.mkdir(os.path.join(*incumbent_path))
                    incumbent_path = []
                    for directory in ['save'] + os.path.normpath(path).split(os.sep):
                        incumbent_path.append(directory)
                        if not os.path.exists(os.path.join(*incumbent_path)):
                            os.mkdir(os.path.join(*incumbent_path))
                
                min_var_portfolio, min_var_obj = min_variance(true_cov)
                min_var_expected_return = np.dot(true_mu, min_var_portfolio)
                
                print('Minimum Variance:', np.round(min_var_obj,4))
                
                print('Expected Return of Min Variance Portfolio:',np.round(min_var_expected_return,4))
                
                v_begin = min_var_obj
                v_end = np.diag(true_cov).max()
                v_array = np.linspace(v_begin, v_end, 10)
                v_array = [float(np.format_float_positional(v_array[i], precision=3, unique=False, fractional=False, trim='k')) for i in range(len(v_array))]
    
                if v_array[0] < min_var_obj:
                    v_array[0] = float(str(v_array[0])[:-1] + str(int(str(v_array[0])[-1])+1))
    
                v_array = v_array[:int(len(v_array)/2)]
                print(v_array)
                
                # if units=='monthly':
                #     v_factor = 1
                # elif units=='daily':
                #     v_factor = 0.1
                
                # v_begin = next(x[1] for x in enumerate([i*2.5*v_factor for i in range(11)]) if x[1] > min_var_obj)
                # v_end = v_begin+(20*v_factor)
                # precision = 2.5*v_factor
                # num_v = (v_end - v_begin) / precision
                # v_array = np.linspace(v_begin, v_end, int(num_v) + 1)
                
                estimated_portfolios = {}
                markowitz_portfolios = {}
                robust_portfolios = {}
                
                print('Computing Estimated, Actual and Robust Portfolios')
    
                for kappa in kappa_list:
                    print('\tOn κ={}'.format(kappa))                                
                    estimated_portfolios[kappa] = {}        
                    robust_portfolios[kappa] = {}
                    for run in range(num_runs):
                        estimated_portfolios[kappa][run] = {} 
                        robust_portfolios[kappa][run] = {}
                        if run % 2500 == 0:
                            print('\t\tOn run ', run)
                        for n in N:
                            estimated_portfolios[kappa][run][n] = {}                        
                            robust_portfolios[kappa][run][n] = {}
                                                    
                            for v in v_array:                                                        
                                    
                                if heuristic:
                                    kappa_run = choose_kappa(mu_hat[run][n],true_cov,Xi,v,lb,ub)
                                    
                                    # Dealing with precision issues by scaling Xi
                                    Xi = Xi*(kappa_run**2)
                                    # display(pd.DataFrame(Xi))
                                    kappa_run = 1
                                    
                                estimated_portfolio, estimated_obj = markowitz(mu_hat[run][n], true_cov, v)
                                estimated_portfolios[kappa][run][n][v] = estimated_portfolio
                                            
                                robust_estimated_portfolio, robust_estimated_obj = robust(mu_hat[run][n], true_cov, Xi, v, kappa_run)
                                robust_portfolios[kappa][run][n][v] = robust_estimated_portfolio                                
    
                print('\nComputing True and Equal-Weight Portfolios')
                                    
                for v in v_array:
                    markowitz_portfolio, markowitz_obj = markowitz(true_mu, true_cov, v)
                    markowitz_portfolios[v] = markowitz_portfolio
                
                if save:
                    with open(os.path.join('save',path,'estimated_portfolios (top{}, N={}, κ={}, ratio={}).json'.format(topx,N,kappa_list,ratio)), 'w') as f: 
                        json.dump(estimated_portfolios, f, ensure_ascii=False, indent=4)
                
                    with open(os.path.join('save',path,'markowitz_portfolios (top{}, N={}, κ={}, ratio={}).json'.format(topx,N,kappa_list,ratio)), 'w') as f: 
                        json.dump(markowitz_portfolios, f, ensure_ascii=False, indent=4)
                
                    with open(os.path.join('save',path,'robust_portfolios (top{}, N={}, κ={}, ratio={}).json'.format(topx,N,kappa_list,ratio)), 'w') as f: 
                        json.dump(robust_portfolios, f, ensure_ascii=False, indent=4)
    
                if save:
                    with open(os.path.join('save',path,'estimated_portfolios (top{}, N={}, κ={}, ratio={}).json'.format(topx,N,kappa_list,ratio))) as f: 
                        estimated_portfolios = json.load(f)
                
                    with open(os.path.join('save',path,'markowitz_portfolios (top{}, N={}, κ={}, ratio={}).json'.format(topx,N,kappa_list,ratio))) as f: 
                        markowitz_portfolios = json.load(f)
                
                    with open(os.path.join('save',path,'robust_portfolios (top{}, N={}, κ={}, ratio={}).json'.format(topx,N,kappa_list,ratio))) as f: 
                        robust_portfolios = json.load(f)
                
                markowitz_frontier = {}
                estimated_frontier = {}
                actual_frontier = {}
                
                robust_estimated_frontier = {}
                robust_actual_frontier = {}
                
                for kappa in estimated_portfolios.keys():
                    estimated_frontier[kappa] = {}
                    actual_frontier[kappa] = {}
                    
                    robust_estimated_frontier[kappa] = {}
                    robust_actual_frontier[kappa] = {}
                    
                    for run in estimated_portfolios[kappa].keys():
                        estimated_frontier[kappa][run] = {}
                        actual_frontier[kappa][run] = {}
                        
                        robust_estimated_frontier[kappa][run] = {}
                        robust_actual_frontier[kappa][run] = {}
                        
                        for n in estimated_portfolios[kappa][run].keys():
                            estimated_frontier[kappa][run][n] = {}
                            actual_frontier[kappa][run][n] = {}
                            
                            robust_estimated_frontier[kappa][run][n] = {}
                            robust_actual_frontier[kappa][run][n] = {}
                            
                            for v in estimated_portfolios[kappa][run][n].keys():
                                estimated_frontier[kappa][run][n][v] = np.dot(mu_hat[int(run)][int(n)], np.array(estimated_portfolios[kappa][run][n][v]))
                                actual_frontier[kappa][run][n][v] = np.dot(true_mu, np.array(estimated_portfolios[kappa][run][n][v]))
                                
                                robust_estimated_frontier[kappa][run][n][v] = np.dot(mu_hat[int(run)][int(n)], 
                                                                               np.array(robust_portfolios[kappa][run][n][v]))
                    
                                robust_actual_frontier[kappa][run][n][v] = np.dot(true_mu, np.array(robust_portfolios[kappa][run][n][v]))
                
                for v in markowitz_portfolios.keys():
                    markowitz_frontier[float(v)] = np.dot(true_mu, np.array(markowitz_portfolios[v]))    
                    
                equal_portfolio = np.array([1/num_sectors for i in range(num_sectors)])
                equal_frontier = np.dot(true_mu, equal_portfolio)
                equal_weight_variance = sum(true_cov[i][j] * equal_portfolio[i] * equal_portfolio[j] 
                                            for i in range(num_sectors) for j in range(num_sectors))
                
                mean_actual_frontier = {}
                mean_estimated_frontier = {}
                
                mean_robust_actual_frontier = {}
                mean_robust_estimated_frontier = {}
                for kappa in actual_frontier.keys():
                    mean_actual_frontier[kappa] = {}
                    mean_estimated_frontier[kappa] = {}
                    
                    mean_robust_actual_frontier[kappa] = {}
                    mean_robust_estimated_frontier[kappa] = {}
                    for n in actual_frontier[kappa][run].keys():
                        mean_actual_frontier[kappa][int(n)] = {}
                        mean_estimated_frontier[kappa][int(n)] = {}
                    
                        mean_robust_actual_frontier[kappa][int(n)] = {}
                        mean_robust_estimated_frontier[kappa][int(n)] = {}
                        
                        for v in actual_frontier[kappa][run][n].keys():
                            mean_actual_frontier[kappa][int(n)][float(v)] = np.mean([actual_frontier[kappa][run][n][v] 
                                                                                     for run in actual_frontier[kappa].keys()])
                            mean_estimated_frontier[kappa][int(n)][float(v)] = np.mean([estimated_frontier[kappa][run][n][v] 
                                                                                        for run in estimated_frontier[kappa].keys()])
                            mean_robust_actual_frontier[kappa][int(n)][float(v)] = np.mean([robust_actual_frontier[kappa][run][n][v] 
                                                                                            for run in robust_actual_frontier[kappa].keys()])
                            mean_robust_estimated_frontier[kappa][int(n)][float(v)] = np.mean([robust_estimated_frontier[kappa][run][n][v] 
                                                                                               for run in robust_estimated_frontier[kappa].keys()])
                
                # font = {'size': 20}
                # matplotlib.rc('font', **font)
                # matplotlib.rcParams['mathtext.fontset'] = 'stix'
                # matplotlib.rcParams['font.family'] = 'STIXGeneral'
                
                # for kappa in mean_robust_actual_frontier.keys():
                #     fig, (ax1) = plt.subplots(1,1)
                #     fig.set_size_inches(21, 12)
                    
                #     time_range = list(mean_robust_actual_frontier[kappa].keys())
                #     v_array = list(mean_actual_frontier[kappa][time_range[0]].keys())
                    
                #     x_title = 'Portfolio Returns'
                #     # color_list = ['dodgerblue', 'indianred', 'darkorange', 'mediumseagreen', 'darkviolet', 'midnightblue','fuchsia', 'slategrey']
                #     color_list = ['black'] + list(sns.color_palette("Set1", n_colors = len(time_range)*2)) + ['slategrey']
                    
                #     ax1.plot(v_array, [markowitz_frontier[v] for v in v_array], 
                #              c=color_list[0], marker='o', linestyle='solid', label='True Frontier')
                    
                #     for i in range(0,len(time_range),2):
                #         ax1.plot(v_array, [mean_estimated_frontier[kappa][time_range[i]][v] for v in v_array], c=color_list[1+i],
                #                  marker='x', linestyle='solid', label='Estimated Markowitz Frontier')
                #         ax1.plot(v_array, [mean_actual_frontier[kappa][time_range[i]][v] for v in v_array], c=color_list[1+i],
                #                  marker='x', linestyle='dashed', label='Actual Markowitz Frontier')
                    
                #         ax1.plot(v_array, [mean_robust_estimated_frontier[kappa][time_range[i]][v] for v in v_array], c=color_list[2+i], 
                #                  marker='o', linestyle='solid', label='Estimated Robust Frontier')
                #         ax1.plot(v_array, [mean_robust_actual_frontier[kappa][time_range[i]][v] for v in v_array], c=color_list[2+i], 
                #                  marker='o', linestyle='dashed', label='Actual Robust Frontier')
                    
                #     equal_weight_v_array = [np.round(equal_weight_variance,1)] + [i for i in v_array if i >= equal_weight_variance]    
                #     ax1.plot(equal_weight_v_array, [equal_frontier for v in range(len(equal_weight_v_array))], 
                #              c=color_list[-1], marker='', linestyle='dashed', label='Equal-Weight Frontier')
                    
                #     ax1.set_title(x_title, fontsize=21)
                    
                #     plt.legend(fontsize=18, loc='center left', bbox_to_anchor=(1,0.5))
                #     # plt.annotate('Number of Runs = '+str(num_runs), fontsize=18, xy=(1.09,0.65),xycoords='axes fraction', 
                #     #              bbox=dict(facecolor='none', edgecolor='none', boxstyle='round,pad=0.5'))
                #     # plt.annotate('κ = '+str(kappa), fontsize=18, xy=(1.09,0.7),xycoords='axes fraction', 
                #     #              bbox=dict(facecolor='none', edgecolor='none', boxstyle='round,pad=0.5'))
                
                #     matplotlib.pyplot.xticks(ticks=v_array)
                    
                #     ax1.set_xlabel('Risk (%)', fontsize=19)
                #     ax1.set_ylabel('Expected Portfolio Return (%)', fontsize=19)
                    
                #     ax1.tick_params(axis='both', which='major', labelsize=17)
                #     ax1.tick_params(axis='both', which='minor', labelsize=17)
                
                #     plt.tight_layout()
                    
                #     #  saving the figure. 
                #     if save:        
                #         plt.savefig(os.path.join('plots', path,
                #                                  'averaged_robust_frontier_plot (top{}, N={}, κ={}).pdf'.format(topx,N,kappa)), 
                #                     bbox_inches ='tight')
                        
                #         if data == 'GICS':
                #             ax1.set_ylim([1.2, 2.0])
                #         else:
                #             if units == 'daily':
                #                 ax1.set_ylim([0.04, 0.06])
                #             else:
                #                 ax1.set_ylim([0.9, 1.2])
                #         plt.savefig(os.path.join('plots', path, 'Zoomed',
                #                                  'averaged_robust_frontier_plot (top{}, N={}, κ={}).pdf'.format(topx,N,kappa)), 
                #                     bbox_inches ='tight')
                
                gap_closed = {}
                
                for kappa in mean_robust_actual_frontier.keys():
                    gap_closed[kappa] = {}
                    for n in mean_robust_actual_frontier[kappa].keys():
                        for v in list(mean_robust_actual_frontier[kappa][n].keys())[1:]:
                            robust_expected_return = mean_robust_actual_frontier[kappa][n][v]
                            markowitz_expected_return = mean_actual_frontier[kappa][n][v]        
                            true_expected_return = markowitz_frontier[v]
                            gap_closed[kappa][v] = (robust_expected_return - markowitz_expected_return)/(true_expected_return - markowitz_expected_return)*100
                
                percent_gap_closed_df = pd.DataFrame(gap_closed).T
                display(percent_gap_closed_df)
                
                if save:
                    percent_gap_closed_df.to_csv(os.path.join('plots', path,'averaged_robust_frontier_plot (top{},N={},kappa={},ratio={}).csv'.format(topx,N,kappa_list,ratio)))

Considering GICS data
True Mean:


,0,1,2,3,4,5,6,7,8,9,10
0,1.177,1.511,1.391,1.151,1.289,1.332,1.032,1.726,1.39,1.416,1.014


True Covariance:


,0,1,2,3,4,5,6,7,8,9,10
0,39.467,15.328,9.222,17.256,17.997,18.296,10.578,16.700,21.803,11.611,8.835
1,15.328,28.297,13.578,26.924,22.956,24.931,14.119,26.058,22.964,13.237,5.476
2,9.222,13.578,17.198,15.055,13.905,15.840,10.254,11.174,13.007,13.338,7.370
3,17.256,26.924,15.055,52.492,26.411,27.133,14.250,26.131,29.716,13.643,8.078
4,17.997,22.956,13.905,26.411,26.528,25.398,13.513,23.726,24.327,14.576,7.437
5,18.296,24.931,15.840,27.133,25.398,39.451,14.458,22.606,24.142,17.601,8.495
6,10.578,14.119,10.254,14.250,13.513,14.458,30.009,16.993,12.452,10.015,7.304
7,16.700,26.058,11.174,26.131,23.726,22.606,16.993,50.455,22.041,13.846,2.584
8,21.803,22.964,13.007,29.716,24.327,24.142,12.452,22.041,32.438,13.312,5.912
9,11.611,13.237,13.338,13.643,14.576,17.601,10.015,13.846,13.312,21.559,7.575


Xi:


...

,1.17,1.55,1.93,2.31
heuristic,8.964506,11.607333,-7.452621,-5.100304


lb: 2 ub: 4 ratio: 3
List of Kappas: ['heuristic']
Minimum Variance: 0.7873
Expected Return of Min Variance Portfolio: 0.0431
[0.788, 1.17, 1.55, 1.93, 2.31]
Computing Estimated, Actual and Robust Portfolios
	On κ=heuristic
		On run  0
		On run  2500
		On run  5000
Iteration Depth Exceeded.
		On run  7500

Computing True and Equal-Weight Portfolios


,1.17,1.55,1.93,2.31
heuristic,6.961221,11.890269,-4.1413,-4.236986


lb: 3 ub: 5 ratio: 4
List of Kappas: ['heuristic']
Minimum Variance: 0.7873
Expected Return of Min Variance Portfolio: 0.0431
[0.788, 1.17, 1.55, 1.93, 2.31]
Computing Estimated, Actual and Robust Portfolios
	On κ=heuristic
		On run  0
Iteration Depth Exceeded.
		On run  2500
		On run  5000
Iteration Depth Exceeded.
		On run  7500

Computing True and Equal-Weight Portfolios


,1.17,1.55,1.93,2.31
heuristic,5.514432,11.542962,-1.152503,-3.828222


***

In [3]:
# fig, ax = plt.subplots(1, 1)
# fig.set_size_inches(16, 8)

# moving_average_n = 18

# avg_returns = np.mean(returns, axis=1)
# avg_returns_ma = np.array([np.mean(avg_returns[k:k+moving_average_n]) for k in range(returns.shape[0]+1-moving_average_n+1)])
# ax.plot([j for j in range(1,returns.shape[0]+1)], avg_returns, c="dodgerblue", alpha=0.4, label='Monthly Returns')
# ax.plot([j for j in range(len(avg_returns_ma))], avg_returns_ma, c="indianred", label='Moving Average of Monthly Returns')
# ax.set_title('Fama-French Average Monthly Returns 1927-2024 with Moving Average', fontsize=19)

# ax.set(xlabel='Month', ylabel='Monthly Return')
# # ax.set_xticks([j*12 for j in range(0,returns.shape[0]+1)], [2024-50+i for i in range(0,51)])
# plt.xticks(rotation=45)

# font = {'size': 20}
# matplotlib.rc('font', **font)
# matplotlib.rcParams['mathtext.fontset'] = 'stix'
# matplotlib.rcParams['font.family'] = 'STIXGeneral'

# fig.tight_layout()


***

### Notes June 12th

* Try $\Xi = V^T F V$ -- full matrix and then just diagonal
* Try Ledoit-Wolf for $\Xi$

### Notes July 8th

* Base scenario as identity matrix and then try to find a more reasonable $\Xi$ given that $\Sigma$ is known and that we can extract some information out of it as a comparison. $\Xi = I, \text{diag}(\sigma_i^2), \text{diag}(\frac{1}{\sigma_i^2})$ Maybe the metric as gap closed is good.

* Normalize so that we have $\frac{1}{\sigma^2}$ on the best half of the assets and then $\frac{1}{\text{avg of all}(\sigma^2)}$ for the the bottom half.

*  Once we figure out the best choices for $\Xi$, we want to use this on the time-varying, real-data stream so show how it works practically.

* Look at splitting up the 30 years into 3 decades and use them as independent datasets to essentially triple our data.

### Notes July 11th

* The current experiments seem promising. Once we figure out the best choices for $\Xi$, we want to use this on the time-varying, real-data stream so show how it works practically. See board picture for July 11th.


* ~Look into just $\frac{1}{\sigma^2}$ for all assets or maybe $\frac{2}{\text{avg of all}(\sigma^2)}$.~

* Look at splitting up the 30 years based on the market trends for all assets. This would give us more data and reinforce the assumption that $\Sigma$ is fixed and known. Treat them as independent datasets.
    * 1986 - 1992
    * 1992 - 2000
    * 2000 - 2002
    * 2002 - 2005
    * 2005 - 2008
    * **2008 - 2018**
    * 2018 - 2024

* Look into confidence interval for the final dataset we want to show.

### Notes July 15th

* Try out $\frac{1}{\sigma_i}$ and $\frac{1}{\sigma_i^3}$ depending on the results

### Notes July 26th

* Think about a rule of thumb for picking $\kappa$. It would be nice to have it as a function of the risk $v$ and in the same magnitude as the first term $\hat \mu \mathbf{x}$
    * We have looked at $\kappa = \frac{n \bar \mu}{2 \sqrt{\sum_{i=1}^{n}\xi_i}}$, where $\bar \mu = \frac{1}{n} \sum_{i=1}^{n} |\hat \mu_i|$. However, this was not perfect
* **See what the magnitude of the first term and second term actually are in each run -- maybe average it over the many runs**

### Notes July 29th
* Picking $\kappa$
    1. Compute $\hat \mu \bar x$ where $\bar x$ is the equal weight portfolio
    2. Compute $\sqrt{\bar {\bar x}^\top \Xi \bar {\bar x}}$ where $\bar {\bar x} = (\frac{1}{2}, \frac{1}{2(n-1)},\ldots)$ where the $\xi_1$ is the smallest.
    3. Pick $\kappa = \frac{\hat \mu^\top \bar x}{2 \sqrt{\bar {\bar x}^\top \Xi \bar {\bar x}}}$
    4. Find $x^R$ with this $\kappa$
    5. Stop if $1 \leq \frac{\hat \mu^\top x^R}{2 \sqrt{{x^R}^\top \Xi x^R}} \leq 3$
    6. Else $\kappa \leftarrow \frac{\hat \mu^\top x^R}{2 \sqrt{{x^R}^\top \Xi x^R}}$
* Save results for $\frac{1}{\sigma^{10}}$ and explain in a paragraph why this is not great.
* Say that the ratio being $~1-3$ is based on our data and our choice for $N$ from section 2, and that if you're most confidence with the expected returns, you should be increase your ratio. Verify by showing that as we reduce $N$, this ratio increases.

### Notes August 2nd

* Change the array of risks used. Make the upper bound depend on the risk. Maybe the average of the top 20% risks. Maybe just the **largest** actually. Run for $\Xi = I$, $\Xi = \text{diag}(\sigma^2)$, $\Xi = \text{diag}(\frac{1}{\sigma})$,  $\Xi = \text{diag}(\frac{1}{\sigma^2})$, $\Xi = \text{diag}(\frac{1}{\sigma^4})$, $\Xi = \text{diag}(\frac{1}{\sigma^{10}})$.
* We will include the graphs as well as the tables in the paper. In the graphs, let's include the minimum variance
* Instead of a full grid search for $\kappa$, we could have a more compact table in the paper which only uses the $\kappa$ we found in our heuristic algorithm. We reason why this ratio of 1-3 is good for our dataset -- if you're most confidence with the expected returns, you should be increase your ratio.
* Try to have a clear statement on what happens with low risk and what happens with high risk. Maybe a breakpoint with the variance of the equal weight portoflio. There would be a different optimum $\Xi$ depending on the risk you look at. Show that for a low risk, we want a smaller exponent on $\sigma$, for medium risk we have a medium exponent, and for high risk, a large exponent. 

### Notes August 6th

* Finish up draft of the paper up to the simulated experiments
* See what happens with the real data -- Look at the board from July 11 but have $\Sigma$ be fixed over 30 years
* Maybe shift the weight to be between $2-4$ in the search kappa problem.

### Notes August 13th

* Re run real data experiments with the Identity.
* Do a grid search on the ratio for the $\kappa$ heuristic. -- Do this for just the middle range of the variances because when we look at high risk, the constraint is likely not even tight, and the optimization problem is uninteresting. Go up to half the current v_array
    * We need to give a better justification for the ratio. Maybe need to have a ratio for each choice of the risk or for each choice of $\Xi$

### Notes August 20th

* Try $\frac{1}{\sigma^{15}}$ for the estimation error matrix
* Maybe $\frac{1}{\sigma^{14}}$ or $\frac{1}{\sigma^{12}}$ if we see numerical issues.

### Notes Augst 22nd

* Do the time varying experiments for the $\Xi$ sections